In [1]:
import black


def print_res(old_d:dict,result, just_ans=True):
    d=old_d.copy()
    d['tools']=eval(d['tools'])
    if just_ans:
        del d['tools']
    d['answers']=eval(d['answers'])
    print("-"*12,end="\n\n\n")
    print(black.format_str(str(d), mode=black.Mode()))
    print("-"*12,end="\n\n\n")
    print(result)

In [3]:
import pandas as pd

df = pd.read_json("hf://datasets/Salesforce/xlam-function-calling-60k/xlam_function_calling_60k.json")
# df=pd.read_json("./xlam_function_calling_60k.json")

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import  StrOutputParser 
from prompts import main_prompt
prompt=ChatPromptTemplate.from_messages([("system",main_prompt),('user','{query}')])
llm=prompt|ChatOpenAI(model='gpt-4o-mini',temperature=0)|StrOutputParser()

In [8]:
d=df.iloc[6].to_dict()
result=llm.invoke(
    d
)
print_res(d,result)


------------


{
    "id": 6,
    "query": "Identify the ID of 'Pulp Fiction' and search for torrents related to it.",
    "answers": [
        {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
        {
            "name": "search_torrents",
            "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
        },
    ],
}

------------


```shonqol
movie_id = get_id("Pulp Fiction");
torrents = search_torrents(concat("Pulp Fiction", " torrent"), 40);
```


In [9]:
d=df.iloc[9].to_dict()
result=llm.invoke(
    d
)
print_res(d,result)


------------


{
    "id": 9,
    "query": "Is 'https://www.apple.com' available in the Wayback Machine on September 9, 2015?",
    "answers": [
        {
            "name": "availability",
            "arguments": {"url": "https://www.apple.com", "timestamp": "20150909"},
        }
    ],
}

------------


```shonqol
url = "https://www.apple.com";
timestamp = "20150909";
availabilityStatus = availability(url, timestamp);
print(availabilityStatus);
```


In [10]:
d=df.iloc[103].to_dict()
result=llm.invoke(
    d
)
print_res(d,result,just_ans=False)


------------


{
    "id": 103,
    "query": "What is the recommended daily calorie intake and macronutrient distribution for a 30-year-old male, weighing 75 kg, 180 cm tall, moderately active, aiming to maintain weight?",
    "answers": [
        {
            "name": "calculate_calorie_intake",
            "arguments": {
                "weight_kg": 75,
                "height_cm": 180,
                "age": 30,
                "sex": "male",
                "activity_level": 3,
                "goal": "maintain",
            },
        }
    ],
}

------------


```shonqol
calorie_intake = calculate_calorie_intake(
    weight_kg = 75,
    height_cm = 180,
    age = 30,
    sex = "male",
    activity_level = 3,
    goal = "maintain"
);
```


In [13]:
d=df.iloc[586].to_dict()
result=llm.invoke(
    d
)
print_res(d,result,just_ans=True)


------------


{
    "id": 586,
    "query": "I'm curious about the neuronal activity rate when the synaptic input rate is 100 per second, with a synaptic weight of 0.5 and a decay constant of 0.05. Also, could you check if 17 is a prime number and solve the quadratic equation 2x^2 - 3x + 1 = 0?",
    "answers": [
        {
            "name": "neuronal_activity_rate",
            "arguments": {
                "synaptic_input_rate": 100,
                "synaptic_weight": 0.5,
                "decay_constant": 0.05,
            },
        },
        {"name": "is_prime", "arguments": {"num": 17}},
        {"name": "solve_quadratic", "arguments": {"a": 2, "b": -3, "c": 1}},
    ],
}

------------


Here is the Shonqol code that calculates the neuronal activity rate, checks if 17 is a prime number, and solves the quadratic equation \(2x^2 - 3x + 1 = 0\):

```shonqol
neuronal_activity = neuronal_activity_rate(100, 0.5, 0.05);
is_seventeen_prime = is_prime(17);
quadratic_roots = solve_quad

In [18]:
d=df.iloc[998].to_dict()
result=llm.invoke(
    d
)
print_res(d,result,)


------------


{
    "id": 998,
    "query": "List all tokens on the Ethereum network.",
    "answers": [{"name": "tokens", "arguments": {"network": "ethereum"}}],
}

------------


```shonqol
tokensList = tokens(network="ethereum");
print(tokensList);
```


In [30]:
import json
for _, row in df.iterrows():

    v= json.loads(str(row['answers'])) 
    if not isinstance(v,list): continue
    if len(v)>3:
        d=row.to_dict()
        break
result=llm.invoke(
        d
    )
print_res(d,result,just_ans=True)

------------


{
    "id": 107,
    "query": "I'm conducting a research study and need the men's marks for points values of 10, 20, 30, 40, and 50 to observe the trend in the lower range.",
    "answers": [
        {"name": "marks_men_points", "arguments": {"points": 10}},
        {"name": "marks_men_points", "arguments": {"points": 20}},
        {"name": "marks_men_points", "arguments": {"points": 30}},
        {"name": "marks_men_points", "arguments": {"points": 40}},
        {"name": "marks_men_points", "arguments": {"points": 50}},
    ],
}

------------


```shonqol
marks_10 = marks_men_points(10);
marks_20 = marks_men_points(20);
marks_30 = marks_men_points(30);
marks_40 = marks_men_points(40);
marks_50 = marks_men_points(50);

trend_data = concat(
    concat("Marks for 10 points: ", marks_10), 
    concat("\nMarks for 20 points: ", marks_20),
    concat("\nMarks for 30 points: ", marks_30),
    concat("\nMarks for 40 points: ", marks_40),
    concat("\nMarks for 50 points: ", ma

In [ ]:
d=df.iloc[6].to_dict()
result=llm.invoke(
    d
)
print_res(d,result)


------------


{
    "id": 6,
    "query": "Identify the ID of 'Pulp Fiction' and search for torrents related to it.",
    "answers": [
        {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
        {
            "name": "search_torrents",
            "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
        },
    ],
}

------------


```shonqol
movie_id = get_id("Pulp Fiction");
torrents = search_torrents(concat("Pulp Fiction", " torrent"), 40);
```
